Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
# make a compy of train in case it gets corrupted
train_cp = train.copy()

In [4]:
import numpy as np

def engineerFeatures(df):
    
    df["date_recorded_datetime"] = pd.to_datetime(df["date_recorded"])

    df["date_recorded_epoch"] = df["date_recorded_datetime"].astype(np.int64)
    df["date_recorded_month"] = df["date_recorded_datetime"].apply(lambda x: x.month)
    df["date_recorded_year"] = df["date_recorded_datetime"].apply(lambda x: x.year).astype(np.object)

    df["construction_year_na"] = (df["construction_year"] == 0).apply(lambda x: 1 if x else 0)

    df["age"] = df["date_recorded_year"] - df["construction_year"].apply(lambda x: x if x > 0 else 3000)
    df["age"] = df["age"].apply(lambda x: x if x > 0 else df["age"].median())

    df["nan_count"] = df.replace({"unknown": np.NaN}).isnull().sum(axis=1)
    
    df["age_brackets"] = pd.cut(df["age"], [0, 1, 4, 16, 999], labels=[str(x) for x in range(4)]).astype(np.object)
    for col in ["funder", "wpt_name", "region", "management", "source", "installer"]:
        df["age_brackets_by_"+col] = df["age_brackets"] + "_" + df[col]
        

engineerFeatures(train)
engineerFeatures(test)

engineered = train.copy()

In [5]:
engineered.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group', 'date_recorded_datetime',
       'date_recorded_epoch', 'date_recorded_month', 'date_recorded_year',
       'construction_year_na', 'age', 'nan_count', 'age_brackets',
       'age_brackets_by_funder', 'age_brackets_by_wpt_name',
       'age_brackets_by_region', 'age_brackets_by_management',
       'age_brackets_by

In [10]:
# re-import everything so we have it in one place
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_transformer

from sklearn.feature_selection import SelectKBest

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score

train = engineered.copy()

target = "status_group"

bad_cols = [
    "id",
    "scheme_name",
    "date_recorded",
    "date_recorded_datetime",
    "construction_year",
    "management_group",
    "payment_type",
    "quantity_group",
    "source_type",
    "waterpoint_type_group",
    "extraction_type_group",
    "extraction_type_class",
    "region_code",
    "district_code",
    "wpt_name",
    "management",
]

features = train.columns.drop([target] + bad_cols)


In [11]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

class CustomImputer(SimpleImputer):

    def __init__(self):
        self.__col_to_reg = {}
    

    def fit(self, *args, **kwargs):

        df = args[0]
        self.__col_to_reg = {}

        for col in df.columns:
            features = df.columns.drop(col)
            target = col

            reg = DecisionTreeRegressor()
            reg.fit(df[features], df[col])

            self.__col_to_reg[col] = reg
        
        return df
        

    def transform(self, *args, **kwargs):

        df = args[0]
        df = df.copy()

        for col in df.columns[df.isna().any()]:

            features = df.columns.drop(col)

            reg = self.__col_to_reg[col]
            df[col] = reg.predict(df[features])*df[col].isnull().astype(np.int64) + df[col]*(~(df[col].isnull()).astype(np.int64))
        
        return df

    
    def fit_transform(self, *args, **kwargs):
        df = args[0]

        self.fit(df)
        return self.transform(df)


In [12]:
def runPredictions():
    """
        Split the data, make a pipeline, fit the pipeline,
        make predictions, print the feature importance

        @returns: pipeline
    """
    global train, features, score, X_val, y_val

    high_card = []
    low_card = []

    for col in train[features].select_dtypes(include="object"):
        if len(train[col].value_counts()) > 50:
            high_card.append(col)
        else:
            low_card.append(col)

    # split data
    trn, val = train_test_split(train)

    X_train = trn[features]
    y_train = trn[target]

    X_val = val[features]
    y_val = val[target]

    # make the pipeline
    pipeline = make_pipeline(
        make_column_transformer(
            (ce.OrdinalEncoder(), high_card),
            (ce.OneHotEncoder(), low_card),
        ),
        SimpleImputer(),
        StandardScaler(),
        RandomForestClassifier(random_state=511, n_jobs=-1, n_estimators=120),
    )

    pipeline.fit(X_train, y_train)

    # make predictions
    y_pred = pipeline.predict(X_val)
    score = accuracy_score(y_pred, y_val)
    print(f"""
        Model accuracy score: {score}
    """)

    # make visualizations
    model = pipeline.named_steps["randomforestclassifier"]
    #encoder = pipeline.named_steps["ordinalencoder"]
    #encoded_cols = encoder.transform(X_val).columns
    #importances = pd.Series(model.feature_importances_, encoded_cols)

    #fig, ax = plt.subplots(figsize=(10, 40))
    #importances.sort_values().plot.barh(color="gray", ax=ax)
    #fig.show()

    return pipeline

In [13]:
pipeline = runPredictions()


        Model accuracy score: 0.7857239057239057
    


In [14]:
def runPredictions():
    """
        Split the data, make a pipeline, fit the pipeline,
        make predictions, print the feature importance

        @returns: pipeline
    """
    global train, features, score, X_val, y_val

    # split data
    trn, val = train_test_split(train)

    X_train = trn[features]
    y_train = trn[target]

    X_val = val[features]
    y_val = val[target]

    # make the pipeline
    pipeline = make_pipeline(
        ce.OrdinalEncoder(),
        CustomImputer(),
        StandardScaler(),
        RandomForestClassifier(random_state=511, n_jobs=-1, n_estimators=120),
    )

    pipeline.fit(X_train, y_train)

    # make predictions
    y_pred = pipeline.predict(X_val)
    score = accuracy_score(y_pred, y_val)
    print(f"""
        Model accuracy score: {score}
    """)

    # make visualizations
    model = pipeline.named_steps["randomforestclassifier"]
    #encoder = pipeline.named_steps["ordinalencoder"]
    #encoded_cols = encoder.transform(X_val).columns
    #importances = pd.Series(model.feature_importances_, encoded_cols)

    #fig, ax = plt.subplots(figsize=(10, 40))
    #importances.sort_values().plot.barh(color="gray", ax=ax)
    #fig.show()

    return pipeline

In [15]:
pipeline = runPredictions()


        Model accuracy score: 0.8034343434343434
    


In [ ]:
# use RandomSearchCv

from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_regression, f_classif, SelectKBest

trn, val = train_test_split(train)

X_train = trn[features]
y_train = trn[target].replace({"functional": 0, "functional needs repair": 1, "non functional": 2})

X_val = val[features]
y_val = val[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    CustomImputer(),
    StandardScaler(),
    SelectKBest(f_regression),
    RandomForestClassifier(n_jobs=-1),
)

param_distributions = {
    "selectkbest__k": range(1, len(X_train.columns)),
    "randomforestclassifier__n_estimators": range(80, 500),
}


search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=120,
    cv=5,
    scoring="neg_mean_absolute_error",
    verbose=10,
    return_train_score=True,
    n_jobs=-1,
)

search.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.0min


In [23]:
search.best_params_

{'selectkbest__k': 32, 'randomforestclassifier__n_estimators': 497}